<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#查詢會計科目" data-toc-modified-id="查詢會計科目-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>查詢會計科目</a></span></li><li><span><a href="#[以合併為主簡表(累計)]-欄位選擇" data-toc-modified-id="[以合併為主簡表(累計)]-欄位選擇-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>[以合併為主簡表(累計)] 欄位選擇</a></span></li><li><span><a href="#篩選-2020/1/1-還活著的公司，且上市時間大於7年公司，並去除銀行證券業" data-toc-modified-id="篩選-2020/1/1-還活著的公司，且上市時間大於7年公司，並去除銀行證券業-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>篩選 2020/1/1 還活著的公司，且上市時間大於7年公司，並去除銀行證券業</a></span></li><li><span><a href="#最近年度股東權益報酬率＞平均值(市場及產業)" data-toc-modified-id="最近年度股東權益報酬率＞平均值(市場及產業)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>最近年度股東權益報酬率＞平均值(市場及產業)</a></span></li><li><span><a href="#3年平均股東權益報酬率＞15%" data-toc-modified-id="3年平均股東權益報酬率＞15%-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>3年平均股東權益報酬率＞15%</a></span></li><li><span><a href="#最近年度毛利率＞產業平均值" data-toc-modified-id="最近年度毛利率＞產業平均值-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>最近年度毛利率＞產業平均值</a></span></li><li><span><a href="#7年內市值增加值/7年內保留盈餘增加值&gt;1" data-toc-modified-id="7年內市值增加值/7年內保留盈餘增加值>1-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>7年內市值增加值/7年內保留盈餘增加值&gt;1</a></span></li><li><span><a href="#(最近年度自由現金流量/7年前自由現金流量)-1-&gt;-=-1" data-toc-modified-id="(最近年度自由現金流量/7年前自由現金流量)-1->-=-1-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>(最近年度自由現金流量/7年前自由現金流量)-1 &gt; = 1</a></span></li><li><span><a href="#目前市值/未來10年自由現金流量折現值&lt;1" data-toc-modified-id="目前市值/未來10年自由現金流量折現值<1-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>目前市值/未來10年自由現金流量折現值&lt;1</a></span></li><li><span><a href="#投組" data-toc-modified-id="投組-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>投組</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tejapi

In [2]:
tejapi.ApiConfig.api_key = 'Your Key'
tejapi.ApiConfig.ignoretz = True

# 查詢會計科目

In [3]:
data = tejapi.get('TWN/AIACC_INFO_C')
data

# 輸出成 Excel
# data.to_excel('會計科目表.xlsx')

,fin_type,ord,acct_code,cdesc,acct_type
None,,,,,
0,B,1,0112,現金及約當現金,A
1,B,2,0130,應收帳款及票據,A
2,B,3,013D,應收保費,A
3,B,4,0160,其他應收款,A
4,B,5,017O,應收款項,A
...,...,...,...,...,...
4570,S,855,R165,股利殖利率,Z
4571,S,856,R166,現金股利率,Z
4572,S,857,MV,季底普通股市值,Z


# [以合併為主簡表(累計)] 欄位選擇

In [4]:
data = tejapi.get('TWN/AIM1A',
                  coid = '1101',
                  mdate={'gt':'2018-01-01','lt':'2019-12-31'},
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R103','R105','MV','2341','R69B']},
                  chinese_column_name=True,
                  paginate=True)
data

,公司代碼,財報年月,ROE(A)－稅後,營業毛利率,季底普通股市值,保留盈餘,自由現金流量(D)
None,,,,,,,
0,1101,2018-03-01,1.80,23.95,155209904.0,51870788.0,9005775.0
1,1101,2018-06-01,6.63,27.93,179839656.0,48333975.0,4465227.0
2,1101,2018-09-01,10.07,27.51,209941225.0,54267913.0,6888747.0
3,1101,2018-12-01,12.94,26.96,181846900.0,61588761.0,6492976.0
4,1101,2019-03-01,2.08,25.59,210707434.0,65540562.0,6593029.0
5,1101,2019-06-01,6.00,27.87,235226117.0,52320440.0,15427641.0
6,1101,2019-09-01,9.55,29.16,216711793.0,58857987.0,20538786.0
7,1101,2019-12-01,12.54,29.25,238847550.0,65626033.0,18856664.0


# 篩選 2020/1/1 還活著的公司，且上市時間大於7年公司，並去除銀行證券業

- 交易目標，在 2020/1/1 找出投資標的，持有至 2020/12/31

In [5]:
comp_data = tejapi.get('TWN/AIND',
                   elist_day1 ={'lt':'2013-01-01'},
                   ind = {'ne':('28', '30')},
                   mkt = 'TSE',
                   opts = {'columns':['coid', 'elist_day1', 
                           'ind']},
                   chinese_column_name=True,
                   paginate=True
                  )

In [6]:
comp_list = list(comp_data['公司簡稱'])

In [7]:
comp_data

,公司簡稱,最近上市日,TEJ 產業別
None,,,
0,1101,1962-02-09,11
1,1102,1962-06-18,11
2,1103,1969-11-27,11
3,1104,1971-02-02,11
4,1108,1990-06-06,11
...,...,...,...
739,9943,2002-08-26,27
740,9944,2002-08-26,99
741,9945,1992-04-30,99


In [8]:
len(comp_list)

744

# 最近年度股東權益報酬率＞平均值(市場及產業)

In [9]:
ROE_data = tejapi.get('TWN/AIM1A',
                  coid = comp_list,
                  mdate= '2019-12-01',
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R103']},
                  chinese_column_name=True,
                  paginate=True)
ROE_data

,公司代碼,財報年月,ROE(A)－稅後
None,,,
0,1101,2019-12-01,12.54
1,1102,2019-12-01,13.54
2,1103,2019-12-01,3.67
3,1104,2019-12-01,6.35
4,1108,2019-12-01,1.07
...,...,...,...
739,9943,2019-12-01,19.40
740,9944,2019-12-01,-1.37
741,9945,2019-12-01,15.89


 - 產業別列表

In [10]:
industry_list = comp_data["TEJ 產業別"].unique().tolist()

In [11]:
len(industry_list)

19

In [12]:
industry_data = tejapi.get('TWN/AIND',
                    ind = industry_list,
                    mkt = 'TSE',
                    opts = {'columns':['coid', 'ind']},      
                    chinese_column_name=True,
                    paginate=True
                   ).reset_index(drop=True)

In [13]:
industry_data

,公司簡稱,TEJ 產業別
0,1101,11
1,1102,11
2,1103,11
3,1104,11
4,1108,11
...,...,...
925,9944,99
926,9945,99
927,9946,25
928,9955,99


- 市場字典

In [14]:
industry_dict = {}
for i in industry_list:
    industry_dict[i] = industry_data[industry_data['TEJ 產業別'] == i]['公司簡稱'].tolist()

In [15]:
industry_dict['13']

['1301',
 '1303',
 '1304',
 '1305',
 '1307',
 '1308',
 '1309',
 '1310',
 '1312',
 '1313',
 '1314',
 '1315',
 '1321',
 '1323',
 '1324',
 '1325',
 '1326',
 '1337',
 '1340',
 '1341',
 '4306']

- 市場ROE

In [16]:
industry_roe = {}
for i in industry_list:
    data = tejapi.get('TWN/AIM1A',
                  coid = industry_dict[i],
                  mdate='2019-12-01',
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R103']},
                  chinese_column_name=True,
                  paginate=True).reset_index(drop=True)
    industry_roe[i] = data.groupby('公司代碼').mean().mean()[0]

In [17]:
industry_roe

{'11': 7.91857142857143,
 '12': 10.709166666666668,
 '13': 5.478571428571428,
 '25': 5.812830188679246,
 '22': 3.788333333333334,
 '14': 4.296428571428571,
 '99': 9.955064935064938,
 '29': 3.595,
 '23': 5.505598086124399,
 '15': 4.549791666666667,
 '16': 7.865333333333333,
 '17': 5.752187499999998,
 '18': -0.2599999999999998,
 '19': 1.945,
 '20': 1.9509374999999995,
 '21': 7.1890909090909085,
 '26': 2.6960869565217394,
 '97': 10.195,
 '27': 7.894705882352942}

- 合併ROE

In [18]:
ROE_data.rename(columns={'公司代碼':'公司簡稱'}, inplace=True)

In [19]:
ROE_data = ROE_data.merge(comp_data, on='公司簡稱')
ROE_data

,公司簡稱,財報年月,ROE(A)－稅後,最近上市日,TEJ 產業別
0,1101,2019-12-01,12.54,1962-02-09,11
1,1102,2019-12-01,13.54,1962-06-18,11
2,1103,2019-12-01,3.67,1969-11-27,11
3,1104,2019-12-01,6.35,1971-02-02,11
4,1108,2019-12-01,1.07,1990-06-06,11
...,...,...,...,...,...
739,9943,2019-12-01,19.40,2002-08-26,27
740,9944,2019-12-01,-1.37,2002-08-26,99
741,9945,2019-12-01,15.89,1992-04-30,99
742,9955,2019-12-01,0.60,2008-01-21,99


- 將產業別替換成產業平均

In [20]:
ROE_data['產業淨值報酬%'] = ROE_data['TEJ 產業別'].apply(lambda x: industry_roe[x])

In [21]:
ROE_data

,公司簡稱,財報年月,ROE(A)－稅後,最近上市日,TEJ 產業別,產業淨值報酬%
0,1101,2019-12-01,12.54,1962-02-09,11,7.918571
1,1102,2019-12-01,13.54,1962-06-18,11,7.918571
2,1103,2019-12-01,3.67,1969-11-27,11,7.918571
3,1104,2019-12-01,6.35,1971-02-02,11,7.918571
4,1108,2019-12-01,1.07,1990-06-06,11,7.918571
...,...,...,...,...,...,...
739,9943,2019-12-01,19.40,2002-08-26,27,7.894706
740,9944,2019-12-01,-1.37,2002-08-26,99,9.955065
741,9945,2019-12-01,15.89,1992-04-30,99,9.955065
742,9955,2019-12-01,0.60,2008-01-21,99,9.955065


In [22]:
set_1 = set(ROE_data[ROE_data['ROE(A)－稅後'] > ROE_data['產業淨值報酬%']]['公司簡稱'])

In [23]:
len(set_1)

382

# 3年平均股東權益報酬率＞15%

IFRS以合併為主簡表(累計)-全產業(TWN/AIM1A)

In [24]:
ROE_data_3Y = tejapi.get('TWN/AIM1A',
                      coid=comp_list,
                      mdate=['2017-12-01','2018-12-01','2019-12-01'],
                      opts={'pivot':True,
                            'columns':['coid', 'mdate', 'R103']},
                      paginate=True,
                      chinese_column_name=True,
                     )

In [25]:
ROE_data_3Y

,公司代碼,財報年月,ROE(A)－稅後
None,,,
0,1101,2017-12-01,6.88
1,1101,2018-12-01,12.94
2,1101,2019-12-01,12.54
3,1102,2017-12-01,4.65
4,1102,2018-12-01,9.77
...,...,...,...
2227,9955,2018-12-01,-14.65
2228,9955,2019-12-01,0.60
2229,9958,2017-12-01,1.86


In [26]:
# 獲得年度ROE
ROE_data_3Y = ROE_data_3Y.groupby('公司代碼').mean()
ROE_data_3Y

,ROE(A)－稅後
公司代碼,
1101,10.786667
1102,9.320000
1103,3.503333
1104,6.880000
1108,-1.930000
...,...
9943,20.000000
9944,0.346667
9945,21.840000


In [27]:
set_2 = set(ROE_data_3Y[ROE_data_3Y['ROE(A)－稅後'] >= 15].index)

In [28]:
len(set_2)

116

# 最近年度毛利率＞產業平均值

In [29]:
data_gm = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate='2019-12-01',
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R105']},
                  paginate=True,
                  chinese_column_name=True,
                 )
data_gm

,公司代碼,財報年月,營業毛利率
None,,,
0,1101,2019-12-01,29.25
1,1102,2019-12-01,26.95
2,1103,2019-12-01,10.54
3,1104,2019-12-01,11.72
4,1108,2019-12-01,8.01
...,...,...,...
739,9943,2019-12-01,55.64
740,9944,2019-12-01,22.16
741,9945,2019-12-01,20.93


In [30]:
# 產業毛利率
industry_gm = {}
for i in industry_dict:
    data = tejapi.get('TWN/AIM1A',
                  coid= industry_dict[i],
                  mdate='2019-12-01',
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R105']},
                  paginate=True,
                  chinese_column_name=True,
                 ).reset_index(drop=True)

    industry_gm[i] = data.groupby('公司代碼').mean().mean()[0]

In [31]:
industry_gm

{'11': 17.85,
 '12': 26.88416666666667,
 '13': 9.698571428571432,
 '25': 17.612452830188683,
 '22': 21.18194444444444,
 '14': 10.269047619047617,
 '99': 29.2664935064935,
 '29': 41.405454545454546,
 '23': 20.58913875598085,
 '15': 22.969375,
 '16': 13.734,
 '17': 33.39111111111111,
 '18': 20.682,
 '19': 16.044999999999998,
 '20': 7.3599999999999985,
 '21': 21.89909090909091,
 '26': 18.27,
 '97': 14.48,
 '27': 41.77352941176471}

In [32]:
data_gm = data_gm.rename(columns={'公司代碼':'公司簡稱'})
data_gm_signal = data_gm.merge(comp_data, on='公司簡稱')
data_gm_signal

,公司簡稱,財報年月,營業毛利率,最近上市日,TEJ 產業別
0,1101,2019-12-01,29.25,1962-02-09,11
1,1102,2019-12-01,26.95,1962-06-18,11
2,1103,2019-12-01,10.54,1969-11-27,11
3,1104,2019-12-01,11.72,1971-02-02,11
4,1108,2019-12-01,8.01,1990-06-06,11
...,...,...,...,...,...
739,9943,2019-12-01,55.64,2002-08-26,27
740,9944,2019-12-01,22.16,2002-08-26,99
741,9945,2019-12-01,20.93,1992-04-30,99
742,9955,2019-12-01,3.72,2008-01-21,99


In [33]:
data_gm_signal['產業營業毛利率'] = data_gm_signal['TEJ 產業別'].apply(lambda x: industry_gm[x])

In [34]:
data_gm_signal

,公司簡稱,財報年月,營業毛利率,最近上市日,TEJ 產業別,產業營業毛利率
0,1101,2019-12-01,29.25,1962-02-09,11,17.850000
1,1102,2019-12-01,26.95,1962-06-18,11,17.850000
2,1103,2019-12-01,10.54,1969-11-27,11,17.850000
3,1104,2019-12-01,11.72,1971-02-02,11,17.850000
4,1108,2019-12-01,8.01,1990-06-06,11,17.850000
...,...,...,...,...,...,...
739,9943,2019-12-01,55.64,2002-08-26,27,41.773529
740,9944,2019-12-01,22.16,2002-08-26,99,29.266494
741,9945,2019-12-01,20.93,1992-04-30,99,29.266494
742,9955,2019-12-01,3.72,2008-01-21,99,29.266494


In [35]:
set_3 = set(data_gm_signal[data_gm_signal['營業毛利率'] > data_gm_signal['產業營業毛利率']]['公司簡稱'])

In [36]:
len(set_3)

345

# 7年內市值增加值/7年內保留盈餘增加值>1

In [37]:
MV_RE_data = tejapi.get('TWN/AIM1A',
                coid = comp_list,
                mdate= ['2019/12/01','2013/03/01'],
                opts={'pivot':True,
                        'columns':['coid', 'mdate','MV','2341']},
                chinese_column_name=True,
                paginate=True
               ).reset_index(drop=True)
MV_RE_data

,公司代碼,財報年月,季底普通股市值,保留盈餘
0,1101,2013-03-01,138456563.0,45521147.0
1,1101,2019-12-01,238847550.0,65626033.0
2,1102,2013-03-01,117766961.0,88960768.0
3,1102,2019-12-01,161181384.0,108564355.0
4,1103,2013-03-01,10215288.0,8044970.0
...,...,...,...,...
1476,9945,2019-12-01,45347036.0,39901502.0
1477,9955,2013-03-01,2584684.0,224981.0
1478,9955,2019-12-01,1950631.0,-261304.0
1479,9958,2013-03-01,1706625.0,397410.0


In [38]:
MV_RE_data = MV_RE_data.pivot_table(index='公司代碼', columns='財報年月').reset_index()

In [39]:
MV_RE_data

公司代碼                保留盈餘                                 季底普通股市值  \
財報年月       2013-03-01 00:00:00 2019-12-01 00:00:00 2013-03-01 00:00:00   
0     1101          45521147.0          65626033.0         138456563.0   
1     1102          88960768.0         108564355.0         117766961.0   
2     1103           8044970.0          10660775.0          10215288.0   
3     1104           6765931.0          11013644.0          10779454.0   
4     1108            199978.0            269466.0           2934351.0   
..     ...                 ...                 ...                 ...   
739   9943           1984164.0           2724144.0           6143119.0   
740   9944           1347554.0           1456085.0           1777426.0   
741   9945           8729008.0          39901502.0          62339006.0   
742   9955            224981.0           -261304.0           2584684.0   
743   9958            397410.0           1086259.0           1706625.0   

                          
財報年月 2019-12-01 00:00:00  
0            238847550.0  
1            161181384.0  
2             17316333.0  
3             12810736.0  
4              3391704.0  
..                   ...  
739           10503702.0  
740            1898462.0  
741           45347036.0  
742            1950631.0  
743           15147127.0  

[744 rows x 5 columns]

In [40]:
MV_RE_data['保留盈餘增加值'] = MV_RE_data.iloc[:,2] - MV_RE_data.iloc[:,1]
MV_RE_data['市值增加值'] = MV_RE_data.iloc[:,4] - MV_RE_data.iloc[:,3]


In [41]:
MV_RE_data

公司代碼                保留盈餘                                 季底普通股市值  \
財報年月       2013-03-01 00:00:00 2019-12-01 00:00:00 2013-03-01 00:00:00   
0     1101          45521147.0          65626033.0         138456563.0   
1     1102          88960768.0         108564355.0         117766961.0   
2     1103           8044970.0          10660775.0          10215288.0   
3     1104           6765931.0          11013644.0          10779454.0   
4     1108            199978.0            269466.0           2934351.0   
..     ...                 ...                 ...                 ...   
739   9943           1984164.0           2724144.0           6143119.0   
740   9944           1347554.0           1456085.0           1777426.0   
741   9945           8729008.0          39901502.0          62339006.0   
742   9955            224981.0           -261304.0           2584684.0   
743   9958            397410.0           1086259.0           1706625.0   

                             保留盈餘增加值        市值增加值  
財報年月 2019-12-01 00:00:00                           
0            238847550.0  20104886.0  100390987.0  
1            161181384.0  19603587.0   43414423.0  
2             17316333.0   2615805.0    7101045.0  
3             12810736.0   4247713.0    2031282.0  
4              3391704.0     69488.0     457353.0  
..                   ...         ...          ...  
739           10503702.0    739980.0    4360583.0  
740            1898462.0    108531.0     121036.0  
741           45347036.0  31172494.0  -16991970.0  
742            1950631.0   -486285.0    -634053.0  
743           15147127.0    688849.0   13440502.0  

[744 rows x 7 columns]

In [42]:
MV_RE_data['指標'] = MV_RE_data['市值增加值'] / MV_RE_data['保留盈餘增加值']

In [43]:
MV_RE_data

公司代碼                保留盈餘                                 季底普通股市值  \
財報年月       2013-03-01 00:00:00 2019-12-01 00:00:00 2013-03-01 00:00:00   
0     1101          45521147.0          65626033.0         138456563.0   
1     1102          88960768.0         108564355.0         117766961.0   
2     1103           8044970.0          10660775.0          10215288.0   
3     1104           6765931.0          11013644.0          10779454.0   
4     1108            199978.0            269466.0           2934351.0   
..     ...                 ...                 ...                 ...   
739   9943           1984164.0           2724144.0           6143119.0   
740   9944           1347554.0           1456085.0           1777426.0   
741   9945           8729008.0          39901502.0          62339006.0   
742   9955            224981.0           -261304.0           2584684.0   
743   9958            397410.0           1086259.0           1706625.0   

                             保留盈餘增加值        市值增加值         指標  
財報年月 2019-12-01 00:00:00                                      
0            238847550.0  20104886.0  100390987.0   4.993363  
1            161181384.0  19603587.0   43414423.0   2.214616  
2             17316333.0   2615805.0    7101045.0   2.714669  
3             12810736.0   4247713.0    2031282.0   0.478206  
4              3391704.0     69488.0     457353.0   6.581755  
..                   ...         ...          ...        ...  
739           10503702.0    739980.0    4360583.0   5.892839  
740            1898462.0    108531.0     121036.0   1.115221  
741           45347036.0  31172494.0  -16991970.0  -0.545095  
742            1950631.0   -486285.0    -634053.0   1.303871  
743           15147127.0    688849.0   13440502.0  19.511536  

[744 rows x 8 columns]

In [44]:
set_4 = set(MV_RE_data[MV_RE_data['指標']>1]['公司代碼'])

In [45]:
len(set_4)

456

# (最近年度自由現金流量/7年前自由現金流量)-1 > = 1

In [46]:
cash_data = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate=['2013-12-01', '2019-12-01'],
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R69B']},
                  paginate=True,
                  chinese_column_name=True,
                 )

In [47]:
cash_data

,公司代碼,財報年月,自由現金流量(D)
None,,,
0,1101,2013-12-01,15642459.0
1,1101,2019-12-01,18856664.0
2,1102,2013-12-01,12973966.0
3,1102,2019-12-01,27257010.0
4,1103,2013-12-01,719211.0
...,...,...,...
1483,9945,2019-12-01,13977275.0
1484,9955,2013-12-01,-388093.0
1485,9955,2019-12-01,65183.0


In [48]:
cash_data = cash_data.pivot_table(index='公司代碼', columns='財報年月')

In [49]:
cash_data

自由現金流量(D)            
財報年月  2013-12-01  2019-12-01
公司代碼                        
1101  15642459.0  18856664.0
1102  12973966.0  27257010.0
1103    719211.0   -339927.0
1104   1042178.0   1216596.0
1108    172819.0      6538.0
...          ...         ...
9943    485458.0   1383108.0
9944     21081.0   -403524.0
9945  30331323.0  13977275.0
9955   -388093.0     65183.0
9958   -175879.0   1596571.0

[744 rows x 2 columns]

In [50]:
cash_data.columns = cash_data.columns.droplevel(0)
cash_data

財報年月,2013-12-01,2019-12-01
公司代碼,,
1101,15642459.0,18856664.0
1102,12973966.0,27257010.0
1103,719211.0,-339927.0
1104,1042178.0,1216596.0
1108,172819.0,6538.0
...,...,...
9943,485458.0,1383108.0
9944,21081.0,-403524.0
9945,30331323.0,13977275.0


In [51]:
cash_data['指標'] = (cash_data['2019-12-01'] / cash_data['2013-12-01']) - 1

In [52]:
cash_data

財報年月,2013-12-01 00:00:00,2019-12-01 00:00:00,指標
公司代碼,,,
1101,15642459.0,18856664.0,0.205480
1102,12973966.0,27257010.0,1.100900
1103,719211.0,-339927.0,-1.472639
1104,1042178.0,1216596.0,0.167359
1108,172819.0,6538.0,-0.962169
...,...,...,...
9943,485458.0,1383108.0,1.849079
9944,21081.0,-403524.0,-20.141597
9945,30331323.0,13977275.0,-0.539180


In [53]:
set_5 = set(cash_data[cash_data['指標'] > 1].index)

In [54]:
len(set_5)

184

# 目前市值/未來10年自由現金流量折現值<1

In [55]:
import numpy_financial as npf

In [56]:
MV_cash_data = tejapi.get('TWN/AIM1A',
                  coid= comp_list,
                  mdate='2019-12-01',
                  opts={'pivot':True,
                        'columns':['coid', 'mdate', 'R69B', 'MV']},
                  paginate=True,
                  chinese_column_name=True,
                 ).reset_index(drop=True)

In [57]:
MV_cash_data

,公司代碼,財報年月,自由現金流量(D),季底普通股市值
0,1101,2019-12-01,18856664.0,238847550.0
1,1102,2019-12-01,27257010.0,161181384.0
2,1103,2019-12-01,-339927.0,17316333.0
3,1104,2019-12-01,1216596.0,12810736.0
4,1108,2019-12-01,6538.0,3391704.0
...,...,...,...,...
739,9943,2019-12-01,1383108.0,10503702.0
740,9944,2019-12-01,-403524.0,1898462.0
741,9945,2019-12-01,13977275.0,45347036.0
742,9955,2019-12-01,65183.0,1950631.0


In [58]:
def Cashflows(y0):

    y1 = y0 * 1.15
    y2 = y1 * 1.15
    y3 = y2 * 1.15
    y4 = y3 * 1.05
    y5 = y4 * 1.05
    y6 = y5 * 1.05
    y7 = y6 * 1.05
    y8 = y7 * 1.05
    y9 = y8 * 1.05
    y10 = y9 * 1.05
    
    cashflows = np.array([y1,y2,y3,y4,y5,y6,y7,y8,y9,y10])
    
    return cashflows

In [59]:
MV_cash_data['指標'] = MV_cash_data['季底普通股市值'] / MV_cash_data['自由現金流量(D)'].apply(lambda x: npf.npv(0.09, Cashflows(x)))

In [60]:
MV_cash_data

,公司代碼,財報年月,自由現金流量(D),季底普通股市值,指標
0,1101,2019-12-01,18856664.0,238847550.0,1.112956
1,1102,2019-12-01,27257010.0,161181384.0,0.519587
2,1103,2019-12-01,-339927.0,17316333.0,-4.476022
3,1104,2019-12-01,1216596.0,12810736.0,0.925230
4,1108,2019-12-01,6538.0,3391704.0,45.582167
...,...,...,...,...,...
739,9943,2019-12-01,1383108.0,10503702.0,0.667280
740,9944,2019-12-01,-403524.0,1898462.0,-0.413385
741,9945,2019-12-01,13977275.0,45347036.0,0.285068
742,9955,2019-12-01,65183.0,1950631.0,2.629437


In [61]:
set_6 = set(MV_cash_data[(MV_cash_data['指標']<1) & (MV_cash_data['指標']>0)]['公司代碼'])

In [62]:
len(set_6)

204

# 投組

In [63]:
set_1 & set_2 & set_3 & set_4 & set_5 & set_6

{'1216', '1582', '2108', '2373', '6277', '9911', '9943'}

In [65]:
portfolio = tejapi.get('TWN/AIND',
                       coid = ['1216', '1582', '2108', '2373', '6277', '9911', '9943'],
                       opts = {'columns':['coid', 'fnamec']},
                       chinese_column_name=True,
                       paginate=True)
portfolio

,公司簡稱,公司中文全稱
None,,
0,1216,統一企業
1,1582,信錦企業
2,2108,南帝化學工業
3,2373,震旦行
4,6277,宏正自動科技
5,9911,台灣櫻花
6,9943,好樂迪


In [71]:
roi = tejapi.get('TWN/APRCD2',
                 coid = ['1216', '1582', '2108', '2373', '6277', '9911', '9943'],
                 mdate= '2020-12-31',
                 opts = {'columns':['coid','mdate','roi_y']},
                 chinese_column_name=True,
                 paginate=True)
                

In [72]:
roi

,證券代碼,年月日,年報酬率 %
None,,,
0,1216,2020-12-31,-5.7572
1,1582,2020-12-31,0.8665
2,2108,2020-12-31,114.5574
3,2373,2020-12-31,5.3199
4,6277,2020-12-31,7.1101
5,9911,2020-12-31,15.9522
6,9943,2020-12-31,-4.6222


In [74]:
np.mean(roi['年報酬率 %'])

19.060957142857145